# Fine-tuning ruT5 model for chat summarization




In [2]:
# installing huggingface libraries for dataset, models and metrics
!pip install datasets transformers[sentencepiece] sacrebleu
!pip install accelerate -U

!pip install numpy==1.24.3


  Obtaining dependency information for datasets from https://files.pythonhosted.org/packages/e2/cf/db41e572d7ed958e8679018f8190438ef700aeb501b62da9e1eed9e4d69a/datasets-2.15.0-py3-none-any.whl.metadata
  Obtaining dependency information for transformers[sentencepiece] from https://files.pythonhosted.org/packages/12/dd/f17b11a93a9ca27728e12512d167eb1281c151c4c6881d3ab59eb58f4127/transformers-4.35.2-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/123.5 kB ? eta -:--:--
     --------- ----------------------------- 30.7/123.5 kB 1.4 MB/s eta 0:00:01
     --------- ----------------------------- 30.7/123.5 kB 1.4 MB/s eta 0:00:01
     --------------------------- --------- 92.2/123.5 kB 751.6 kB/s eta 0:00:01
     -------------------------------- --- 112.6/123.5 kB 595.3 kB/s eta 0:00:01
     ------------------------------------ 123.5/123.5 kB 605.6 kB/s eta 0:00:00
  Obtaining dependency information for sacrebleu from https://files.pythonhosted.org/packages/df/c0/


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for accelerate from https://files.pythonhosted.org/packages/13/9e/ee987874058f2d93006961f6ff49e0bcb60ab9c26709ebe06bfa8707a4d8/accelerate-0.24.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/261.4 kB ? eta -:--:--
   ---- ----------------------------------- 30.7/261.4 kB 1.3 MB/s eta 0:00:01
   -------------- ------------------------- 92.2/261.4 kB 1.3 MB/s eta 0:00:01
   ------------------------- -------------- 163.8/261.4 kB 1.4 MB/s eta 0:00:01
   ----------------------------- ---------- 194.6/261.4 kB 1.3 MB/s eta 0:00:01
   ---------------------------------------- 261.4/261.4 kB 1.2 MB/s eta 0:00:00



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     ---------------------------------------- 0.0/14.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/14.8 MB 1.3 MB/s eta 0:00:12
     ---------------------------------------- 0.1/14.8 MB 1.1 MB/s eta 0:00:13
     ---------------------------------------- 0.1/14.8 MB 1.1 MB/s eta 0:00:13
     ---------------------------------------- 0.1/14.8 MB 1.1 MB/s eta 0:00:13
     --------------------------------------- 0.1/14.8 MB 476.3 kB/s eta 0:00:31
     --------------------------------------- 0.1/14.8 MB 532.5 kB/s eta 0:00:28
     --------------------------------------- 0.1/14.8 MB 532.5 kB/s eta 0:00:28
      -------------------------------------- 0.2/14.8 MB 540.4 kB/s eta 0:00:28
      -------------------------------------- 0.2/14.8 MB 540.4 kB/s eta 0:00:28
      -------------------------------------- 0.3/14.8 MB 561.1 kB/s eta 0:00:26
      -------------------------------------- 0.3/14.8 MB 561.1 kB/s eta 0:00:26
      -------------------------------------- 0.3/14.


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# Necessary imports
import warnings

from datasets import load_dataset, load_metric
import transformers
import datasets
import random
import pandas as pd
from IPython.display import display, HTML

warnings.filterwarnings('ignore')

c:\Users\Grisha\Desktop\ClownPy\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Base model
As a base model, [ruT5-base](https://huggingface.co/ai-forever/ruT5-base) from [SberDevices](https://sberdevices.ru/) was chosen.

In [4]:
# selecting model checkpoint
model_checkpoint = "ai-forever/ruT5-base"

## Loading the dataset
We are going to fine-tune the model on the [dialogsum-ru](https://huggingface.co/datasets/d0rj/dialogsum-ru) dataset.

In [4]:
# setting random seed for transformers library
transformers.set_seed(42)

# Load the WMT16 dataset
raw_datasets = load_dataset("d0rj/dialogsum-ru")

## Metric

We will use [Sacrebleu](https://huggingface.co/spaces/evaluate-metric/sacrebleu) to compute the [BLEU](https://en.wikipedia.org/wiki/BLEU) metric for evaluation.

In [ ]:
# Load the BLUE metric
metric = load_metric("sacrebleu")

## Preprocessing the data


In [5]:
from transformers import AutoTokenizer

# we will use autotokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

config.json: 100%|██████████| 1.40k/1.40k [00:00<?, ?B/s]
spiece.model: 100%|██████████| 1.00M/1.00M [00:00<00:00, 1.13MB/s]
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [6]:
# prefix for model input
prefix = "summarize:"

In [7]:
max_input_length = 512
max_target_length = 128


def preprocess_function(examples):
    inputs = [prefix + ex for ex in examples["dialogue"]]
    targets = [ex for ex in examples["summary"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [8]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

## Fine-tuning the model

In [9]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
# create a model for the pretrained model
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

pytorch_model.bin:   0%|          | 0.00/892M [00:00<?, ?B/s]

In [10]:
# defining the parameters for training
batch_size = 8
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-samsum",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True,
    report_to='tensorboard',
)

In [11]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [12]:
import numpy as np

# simple postprocessing for text
def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

# compute metrics function to pass to trainer
def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [13]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [14]:
trainer.train()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,1.442600,1.175059,3.334200,16.102000
2,1.312700,1.119819,3.819200,16.192000
3,1.231800,1.074553,3.532600,15.962000
4,1.171800,1.067614,3.896400,16.058000
5,1.135200,1.056109,3.949000,15.982000
6,1.096800,1.051035,4.194900,15.998000
7,1.071000,1.041467,4.091500,15.960000
8,1.037600,1.040787,4.149200,15.994000
9,1.026000,1.044615,4.165000,16.056000
10,1.016100,1.043933,4.296600,16.050000


TrainOutput(global_step=15580, training_loss=1.1778921908377378, metrics={'train_runtime': 9124.4582, 'train_samples_per_second': 13.656, 'train_steps_per_second': 1.707, 'total_flos': 5.693538859739136e+16, 'train_loss': 1.1778921908377378, 'epoch': 10.0})

In [15]:
# saving model
trainer.save_model('best')

In [8]:
from transformers import AutoModelForSeq2SeqLM

# loading the model and run inference for it
model = AutoModelForSeq2SeqLM.from_pretrained('best')
model.eval()
model.config.use_cache = False

In [9]:
def translate(model, inference_request, tokenizer=tokenizer):
    input_ids = tokenizer(inference_request, return_tensors="pt").input_ids
    outputs = model.generate(input_ids=input_ids, max_new_tokens=126)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True, temperature=0))

In [10]:
inference_request = prefix + """
Натали: ты все еще собираешься в Тайланд? 
Джейсон: да, на следующей неделе, как и планировалось. 
Джейсон: почему? 
Натали: не могли бы вы купить мне специи? 
Натали: Я не могу найти здесь ничего похожего 
Натали: а мне они очень нравятся 
Джейсон: конечно, если ты пришлешь мне все имена 
Джейсон: знаешь, я не очень хорошо запоминаю эти 
Натали: конечно, я сфотографирую те, которые у меня есть, чтобы было легче 
Джейсон: отлично, и просто на всякий случай напомни мне об этом через две недели или около того. 
Джейсон: У меня могут быть другие мысли, и я склонен легко что-то забывать :) 
Натали: это не должно быть проблемой :)
"""
translate(model, inference_request,tokenizer)

Джейсон собирается в Тайланд на следующей неделе. Натали просит его купить специи, но Джейсон не может запомнить имена Натали. Натали просит его на всякий случай напомнить ему об этом.


In [13]:
inference_request = prefix + """
Майк: Я должен тебе один! 
Джейк: Вообще-то, ты должен мне 200 :P 
Майк: Я верну тебе деньги, как только смогу. 
Джейк: Какой? 
Майк: Ну, я должен получить зарплату 7-го, так что, наверное, 8-го. 
Джейк: Нет проблем. 
Майк: Еще раз спасибо. Я не знаю, что случилось с деньгами, которые я отложил для этого. 
Джейк: Не беспокойся об этом. В прошлом месяце мне пришлось занять немного денег у родителей. 
Майк: Такая же ситуация? 
Джейк: Более или менее. Задержался с арендной платой, и домовладелец потерял терпение. 
Майк: Исправил ситуацию? 
Джейк: К счастью, да.
"""
translate(model, inference_request,tokenizer)

Майк должен 200 :P, чтобы вернуть 200 :P. Джейк говорит, что в прошлом месяце он задержался с арендной платой, и домовладелец потерял терпение.


In [11]:
inference_request = prefix + """
Джефф: Ты готов к завтрашнему походу? 
Энн: я только что собралась 
Корина: это будет очень сложно? 
Джефф: этот трек довольно тяжелый 
Джефф: пожалуйста, возьми хорошую обувь 
Мария: конечно, какой длины трек? 
Джефф: около 20 км 
Джефф: дай мне проверить 
Мария: спасибо 
Джефф: 21,3 км 
Мария: а это только на маяк? 
Джефф: да 
Мария: так что мы должны сесть на автобус обратно 
Джефф: Думаю, да, возвращаться было бы слишком утомительно. 
Корина: и слишком скучно 
Корина: снова так же 
Джефф: правда 
Джефф: так что я проверю автобусы 
Энн: отлично 
Джефф: да, есть один в 5 вечера. 
Мария: просто отлично! 
Джефф: :)
"""
translate(model, inference_request, tokenizer)

Джефф говорит Энн, что завтрашний поход будет тяжелым, и они поедут на автобусе до маяка.
